<a href="https://colab.research.google.com/github/kridtapon/WFO-Volume-Momentum-Synergy-/blob/main/WFO_Volume_Momentum_Synergy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Commodity Channel Index (CCI)
def calculate_cci(data, period=20):
    """
    Calculate Commodity Channel Index (CCI).
    """
    tp = (data['High'] + data['Low'] + data['Close']) / 3  # Typical Price
    sma = tp.rolling(window=period).mean()  # Simple Moving Average
    mean_deviation = tp.rolling(window=period).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=True)
    cci = (tp - sma) / (0.015 * mean_deviation)
    return cci

# Function to calculate Volume Zone Oscillator (VZO)
def calculate_vzo(data, period=14):
    """
    Calculate Volume Zone Oscillator (VZO).
    """
    close_diff = data['Close'].diff()
    positive_volume = data['Volume'].where(close_diff > 0, 0)
    negative_volume = data['Volume'].where(close_diff < 0, 0)
    total_volume = data['Volume'].rolling(window=period).sum()
    vzo = 100 * (positive_volume.rolling(window=period).sum() - negative_volume.rolling(window=period).sum()) / total_volume
    return vzo

# Define the stock symbol and time period
symbol = 'META'  # Stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate CCI and VZO
df['CCI'] = calculate_cci(df, period=20)
df['VZO'] = calculate_vzo(df, period=14)

# Define Entry and Exit signals based on CCI and VZO
df['Entry'] = (df['CCI'] > -100) & (df['VZO'] > 0)  # Entry when CCI crosses above -100 and VZO > 0
df['Exit'] = (df['CCI'] < 100) & (df['VZO'] < 0)  # Exit when CCI crosses below +100 and VZO < 0

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           124874.090266
Total Return [%]                         24.87409
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                      13763.421537
Max Drawdown [%]                        65.315942
Max Drawdown Duration                      1093.0
Total Trades                                   66
Total Closed Trades                            66
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            33.333333
Best Trade [%]                          51.155418
Worst Trade [%]                        -24.277293
Avg Winning Trade [%]                    9.992463
Avg Losing Trade [%]                    -3.757481


In [25]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate Commodity Channel Index (CCI)
def calculate_cci(data, period=20):
    """
    Calculate Commodity Channel Index (CCI).
    """
    tp = (data['High'] + data['Low'] + data['Close']) / 3  # Typical Price
    sma = tp.rolling(window=period).mean()  # Simple Moving Average
    mean_deviation = tp.rolling(window=period).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=True)
    cci = (tp - sma) / (0.015 * mean_deviation)
    return cci

# Function to calculate Volume Zone Oscillator (VZO)
def calculate_vzo(data, period=14):
    """
    Calculate Volume Zone Oscillator (VZO).
    """
    close_diff = data['Close'].diff()
    positive_volume = data['Volume'].where(close_diff > 0, 0)
    negative_volume = data['Volume'].where(close_diff < 0, 0)
    total_volume = data['Volume'].rolling(window=period).sum()
    vzo = 100 * (positive_volume.rolling(window=period).sum() - negative_volume.rolling(window=period).sum()) / total_volume
    return vzo

# Define the stock symbol and time period
symbol = 'HWM'  # HWM
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with CCI and VZO
def walk_forward_optimization_cci_vzo(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    cci_period_range = range(5, 51)  # Range for CCI periods
    vzo_period_range = range(5, 51)  # Range for VZO periods

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of CCI and VZO parameters
        for params in itertools.product(cci_period_range, vzo_period_range):
            cci_period, vzo_period = params

            # Calculate indicators on the training data
            train_data['CCI'] = calculate_cci(train_data, cci_period)
            train_data['VZO'] = calculate_vzo(train_data, vzo_period)

            # Generate entry and exit signals
            entries = (train_data['CCI'] > -100) & (train_data['VZO'] > 0)
            exits = (train_data['CCI'] < 100) & (train_data['VZO'] < 0)

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (cci_period, vzo_period)

        # Test with the best parameters on the test data
        test_data['CCI'] = calculate_cci(test_data, best_params[0])
        test_data['VZO'] = calculate_vzo(test_data, best_params[1])

        entries = (test_data['CCI'] > -100) & (test_data['VZO'] > 0)
        exits = (test_data['CCI'] < 100) & (test_data['VZO'] < 0)

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)


# Perform walk-forward optimization
results = walk_forward_optimization_cci_vzo(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply CCI and VZO indicators
    yearly_data['CCI'] = calculate_cci(yearly_data, params[0])
    yearly_data['VZO'] = calculate_vzo(yearly_data, params[1])

    # Define entry/exit conditions
    entries = (yearly_data['CCI'] > -100) & (yearly_data['VZO'] > 0)
    exits = (yearly_data['CCI'] < 100) & (yearly_data['VZO'] < 0)

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
<ipython-input-25-9fbabd7b5ca6>:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-25-9fbabd7b5ca6>:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-25-9fbabd7b5ca6>:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret


Walk-Forward Optimization Results:
   Year Best_Params
0  2020     (5, 31)
1  2021     (5, 49)
2  2022     (9, 49)
3  2023    (21, 49)
4  2024     (6, 49)
5  2025    (17, 49)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           469195.356176
Total Return [%]                       369.195356
Benchmark Return [%]                   250.802519
Max Gross Exposure [%]                      100.0
Total Fees Paid                       8620.211149
Max Drawdown [%]                         26.79077
Max Drawdown Duration                       202.0
Total Trades                                   25
Total Closed Trades                            24
Total Open Trades                               1
Open Trade PnL                       66167.441943
Win Rate [%]                            58.333333
Best Trade [%]          

In [26]:
# Filter Test Years
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,350802.519453
Total Return [%],250.802519
Benchmark Return [%],250.802519
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],68.916941
